In [1]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc

from bo_functions_generic import gen_theta_set
from CS2_bo_functions_multi_dim import bo_iter_w_runs
from CS2_bo_functions_multi_dim import find_train_doc_path
from CS2_bo_functions_multi_dim import set_ep

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
#----------------------------------------------

#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

#Set Parameters
#Need to run at a and b, need 2 arrays to test that this will work
Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

Theta_True = Constants[1:3].flatten()
param_dict = {0 : 'a_1', 1 : 'a_2', 2 : 'a_3', 3 : 'a_4',
              4 : 'b_1', 5 : 'b_2', 6 : 'b_3', 7 : 'b_4'}

# print(Theta_True)

d = len(Theta_True)
BO_iters = 3
runs = 2
train_iter = 300
noise_std = 0.1
shuffle_seed = 9
sep_fact = np.linspace(0.9,1,1)
set_lengthscale = None
explore_bias = 1

skip_param_types = 1

obj = np.array(["obj"])
# obj = np.array(["obj","LN_obj"])
# emulator = False
emulator = np.array([False])
sparse_grid = np.array([False])
# sparse_grid = np.array([False,True])
verbose = False
# verbose = True
save_fig = True
# save_fig = False

#Pull Experimental data from CSV
exp_d = 2
n = 15 #Number of experimental data points to use
exp_data_doc = 'Input_CSVs/Exp_Data/d='+str(exp_d)+'/n='+str(n)+'.csv'
exp_data = np.array(pd.read_csv(exp_data_doc, header=0,sep=","))
Xexp = exp_data[:,1:exp_d+1]
Yexp = exp_data[:,-1]
# print(Xexp)
# print(Yexp)
#Define GP Testing space
LHS = True
p=20
bounds = np.array([[-2, -2, -10, -2, -2, -2,  5, -2],
                   [ 2,  2,   0,  2,  2,  2, 15,  2]])
# Theta1 =  np.linspace(-2,2,p) #1x10
# Theta2 =  np.linspace(-2,2,p) #1x10
# theta_mesh = np.array(np.meshgrid(Theta1, Theta2)) #2 Uniform 5x5 arrays 
theta_mesh = gen_theta_set(LHS = LHS, n_points = p, dimensions = d, bounds = bounds)
# print(len(theta_mesh))

print("Runs:", runs)
print("BO Iterations:",BO_iters)
print("%%%%%%%%%%%%%%%%%%%%%%%%%%")
for emul in emulator: 
    sys.stdout.flush()
    obj_use = obj
    print("-------------------")
    print("Emulator?:", emul)
    if emul == True: #Change this based on number of TP for each test
        t = 300
        sparse_grid_use = sparse_grid
    else:
        t = 20
        sparse_grid_use = np.array([sparse_grid[0]]) #Sparse Grid will always be False for 2-Input
        
    for sparse in sparse_grid_use:
#         #Can set ep to 1 for sparse grid if wanted
        if sparse == True:
            obj_use =  np.array(["obj"])
        else:
            obj_use =  obj
#             ep_use = torch.tensor([1]) 
#         else:
#             ep_use = explore_bias
        print("______________________________")
        print("Sparse Grid?:", sparse)  

        for obj_func in obj_use:
            all_data_doc = find_train_doc_path(emul, obj_func, d, t)
            all_data = np.array(pd.read_csv(all_data_doc, header=0,sep=",")) 
            print("Objective Function:", obj_func)
            print("-  -  -  -  -  -  -  -  -  -  -")
            for i in range(len(sep_fact)):
#                 explore_bias = set_ep(emul, obj_func, sparse)
                ep = torch.tensor([float(explore_bias)])
                print("Separation Factor Train/Test:", str(np.round(sep_fact[i],3)))
                print("Lengthscale Set To:", set_lengthscale)
                print("Explore Bias Multiplier:", str(np.round(float(ep),3)))
                results = bo_iter_w_runs(BO_iters,all_data_doc,t,theta_mesh,Theta_True,train_iter,ep, Xexp, Yexp,
                                             noise_std, obj_func, runs, sparse, emul, set_lengthscale, Constants, 
                                             param_dict, verbose, save_fig, shuffle_seed, DateTime, sep_fact = sep_fact[i], 
                                             LHS = LHS, skip_param_types = skip_param_types)
                print("The GP predicts the lowest SSE of", "{:.3e}".format(np.exp(results[3])), "occurs at \u03B8 =", results[2], 
                          "during run", results[1], "at BO iteration", results[0])
                print("At this point, the highest EI occurs at \u03B8 =", results[4])
                print("\n")

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Date and Time:  04-Nov-2022 (11:13:23)
Runs: 2
BO Iterations: 3
%%%%%%%%%%%%%%%%%%%%%%%%%%
-------------------
Emulator?: False
______________________________
Sparse Grid?: False
Objective Function: obj
-  -  -  -  -  -  -  -  -  -  -
Separation Factor Train/Test: 0.9
Lengthscale Set To: None
Explore Bias Multiplier: 1.0
iter 1 complete
iter 2 complete
iter 3 complete


/scratch365/mcarlozo/Toy_Problem/CS2_bo_functions_multi_dim.py:229: RuntimeWarning: invalid value encountered in log
  plot_sse = np.log(list_of_plot_variables[1])


iter 1 complete


/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/matplotlib/contour.py:1486: UserWarning: Warning: converting a masked element to nan.
  self.zmax = float(z.max())
/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/matplotlib/contour.py:1487: UserWarning: Warning: converting a masked element to nan.
  self.zmin = float(z.min())


The GP predicts the lowest SSE of 3.874e+05 occurs at θ = [-0.87168413  0.01739706 -7.70741248 -1.22799022 -0.96168678 -1.08773875
 13.04234808 -0.73684618] during run 2 at BO iteration 2
At this point, the highest EI occurs at θ = [ 1.98471021 -0.98644126 -9.65273484 -0.51860475  0.12190667  0.73255258
 14.89545222  1.71096825]


CPU times: user 11min 37s, sys: 11 s, total: 11min 48s
Wall time: 12min 20s


In [2]:
SSE_matrix = np.array([[0.04776216, 0.31411546, 0.13075961],
 [0.04776171, 0.31287431, 0.100031  ]])
SSE_matrix = np.array([[-0.70103865, -1.56881421, -4.5355792 ],
 [-0.70103866, -1.56881405, -4.27605561]])
print(SSE_matrix[[0,1]])
print(SSE_matrix.shape)
argmin = np.array(np.where(np.isclose(SSE_matrix, np.amin(SSE_matrix),rtol=abs(np.amin(SSE_matrix)*1e-6))==True))
print(argmin)

[[-0.70103865 -1.56881421 -4.5355792 ]
 [-0.70103866 -1.56881405 -4.27605561]]
(2, 3)
[[0]
 [2]]


In [3]:
print(argmin)
# print(SSE_matrix[1,0])
argmin[0]

[[0]
 [2]]


array([0])

In [4]:
# print(argmin[:,1])

In [5]:
if len(argmin) > 1:
    rand_ind = np.random.randint(argmin.shape[1]) #Chooses a random point with the minimum value
    argmin = argmin[:,rand_ind]
    print(argmin)

[0 2]


In [6]:
Theta_Best_matrix = np.random.randint(1, size = (2,3,3))
Theta_Best_all = np.array(Theta_Best_matrix[tuple(argmin)+(Ellipsis,)])
print(Theta_Best_all)

[0 0 0]


In [7]:
w = []
print(len(w))

0
